In [1]:
import os
import pandas as pd
from pathlib import Path
from collections import defaultdict

In [6]:
DATASET_PATH = Path(os.path.abspath('')).parent / 'dataset'
RAW_PATH = DATASET_PATH / 'raw'
SPLITTED_PATH = DATASET_PATH / 'splitted'
DT_COL = 'event_time'
CHUNKSIZE = 1000000


In [7]:
def split_month_by_date(month_df_path: str):
    df_chunks = pd.read_csv(month_df_path, parse_dates=[DT_COL], chunksize=CHUNKSIZE)
    datenums = defaultdict(int)
    for dfch in df_chunks:
        dfch['date'] = dfch[DT_COL].dt.date
        for date, group in dfch.groupby('date'):
            datenums[date] += 1
            batch_num = datenums[date]
            dfch.drop('date', axis=1)
            group.to_parquet(f'{SPLITTED_PATH}/data_{date}_{batch_num}.parquet', index=False)

In [8]:
for fpath in os.listdir(RAW_PATH):
    split_month_by_date(RAW_PATH / fpath)